In [3]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,Lambda,Dense
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np
import matplotlib.pyplot as plt

In [12]:
text = "The speed of transmission is an important point of difference between the two viruses. Influenza has a shorter median incubation period (the time from infection to appearance of symptoms) and a shorter serial interval (the time between successive cases) than COVID-19 virus. The serial interval for COVID-19 virus is estimated to be 5-6 days, while for influenza virus, the serial interval is 3 days. This means that influenza can spread faster than COVID-19. Further, transmission in the first 3-5 days of illness, or potentially pre-symptomatic transmission –transmission of the virus before the appearance of symptoms – is a major driver of transmission for influenza. In contrast, while we are learning that there are people who can shed COVID-19 virus 24-48 hours prior to symptom onset, at present, this does not appear to be a major driver of transmission. The reproductive number – the number of secondary infections generated from one infected individual – is understood to be between 2 and 2.5 for COVID-19 virus, higher than for influenza. However, estimates for both COVID-19 and influenza viruses are very context and time-specific, making direct comparisons more difficult."

In [13]:
sentences = text.split('.')
sentences = [sentence.lower() for sentence in sentences]
sentences

['the speed of transmission is an important point of difference between the two viruses',
 ' influenza has a shorter median incubation period (the time from infection to appearance of symptoms) and a shorter serial interval (the time between successive cases) than covid-19 virus',
 ' the serial interval for covid-19 virus is estimated to be 5-6 days, while for influenza virus, the serial interval is 3 days',
 ' this means that influenza can spread faster than covid-19',
 ' further, transmission in the first 3-5 days of illness, or potentially pre-symptomatic transmission –transmission of the virus before the appearance of symptoms – is a major driver of transmission for influenza',
 ' in contrast, while we are learning that there are people who can shed covid-19 virus 24-48 hours prior to symptom onset, at present, this does not appear to be a major driver of transmission',
 ' the reproductive number – the number of secondary infections generated from one infected individual – is under

In [15]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences)
sequences = tokenizer.texts_to_sequences(sentences)
print(sequences)

[[1, 38, 2, 8, 9, 39, 40, 41, 2, 42, 13, 1, 43, 23], [3, 44, 11, 24, 45, 46, 47, 1, 14, 25, 48, 10, 26, 2, 27, 12, 11, 24, 15, 16, 1, 14, 13, 49, 50, 17, 4, 5, 6], [1, 15, 16, 7, 4, 5, 6, 9, 51, 10, 18, 19, 52, 20, 28, 7, 3, 6, 1, 15, 16, 9, 29, 20], [30, 53, 31, 3, 32, 54, 55, 17, 4, 5], [56, 8, 33, 1, 57, 29, 19, 20, 2, 58, 59, 60, 61, 62, 8, 63, 2, 1, 6, 64, 1, 26, 2, 27, 21, 9, 11, 34, 35, 2, 8, 7, 3], [33, 65, 28, 66, 22, 67, 31, 68, 22, 69, 70, 32, 71, 4, 5, 6, 72, 73, 74, 75, 10, 76, 77, 78, 79, 30, 80, 81, 82, 10, 18, 11, 34, 35, 2, 8], [1, 83, 36, 21, 1, 36, 2, 84, 85, 86, 25, 87, 88, 89, 21, 9, 90, 10, 18, 13, 37, 12, 37], [19, 7, 4, 5, 6, 91, 17, 7, 3], [92, 93, 7, 94, 4, 5, 12, 3, 23, 22, 95, 96, 12, 14, 97, 98, 99, 100, 101, 102], []]


In [17]:
word2idx = tokenizer.word_index
idx2word = tokenizer.index_word
print(word2idx)
print(idx2word)

{'the': 1, 'of': 2, 'influenza': 3, 'covid': 4, '19': 5, 'virus': 6, 'for': 7, 'transmission': 8, 'is': 9, 'to': 10, 'a': 11, 'and': 12, 'between': 13, 'time': 14, 'serial': 15, 'interval': 16, 'than': 17, 'be': 18, '5': 19, 'days': 20, '–': 21, 'are': 22, 'viruses': 23, 'shorter': 24, 'from': 25, 'appearance': 26, 'symptoms': 27, 'while': 28, '3': 29, 'this': 30, 'that': 31, 'can': 32, 'in': 33, 'major': 34, 'driver': 35, 'number': 36, '2': 37, 'speed': 38, 'an': 39, 'important': 40, 'point': 41, 'difference': 42, 'two': 43, 'has': 44, 'median': 45, 'incubation': 46, 'period': 47, 'infection': 48, 'successive': 49, 'cases': 50, 'estimated': 51, '6': 52, 'means': 53, 'spread': 54, 'faster': 55, 'further': 56, 'first': 57, 'illness': 58, 'or': 59, 'potentially': 60, 'pre': 61, 'symptomatic': 62, '–transmission': 63, 'before': 64, 'contrast': 65, 'we': 66, 'learning': 67, 'there': 68, 'people': 69, 'who': 70, 'shed': 71, '24': 72, '48': 73, 'hours': 74, 'prior': 75, 'symptom': 76, 'onset

In [20]:
vocab_size = len(word2idx)+1
emb_size = 100
context_size = 2

targets = []
contexts = []

for sequence in sequences:
  for i in range(context_size,len(sequence)-context_size):
    target = sequence[i]
    context = [sequence[i-2],sequence[i-1],sequence[i+1],sequence[i+2]]
    targets.append(target)
    contexts.append(context)

print(targets)
print(contexts)

[2, 8, 9, 39, 40, 41, 2, 42, 13, 1, 11, 24, 45, 46, 47, 1, 14, 25, 48, 10, 26, 2, 27, 12, 11, 24, 15, 16, 1, 14, 13, 49, 50, 17, 4, 16, 7, 4, 5, 6, 9, 51, 10, 18, 19, 52, 20, 28, 7, 3, 6, 1, 15, 16, 9, 31, 3, 32, 54, 55, 17, 33, 1, 57, 29, 19, 20, 2, 58, 59, 60, 61, 62, 8, 63, 2, 1, 6, 64, 1, 26, 2, 27, 21, 9, 11, 34, 35, 2, 8, 28, 66, 22, 67, 31, 68, 22, 69, 70, 32, 71, 4, 5, 6, 72, 73, 74, 75, 10, 76, 77, 78, 79, 30, 80, 81, 82, 10, 18, 11, 34, 35, 36, 21, 1, 36, 2, 84, 85, 86, 25, 87, 88, 89, 21, 9, 90, 10, 18, 13, 37, 4, 5, 6, 91, 17, 7, 94, 4, 5, 12, 3, 23, 22, 95, 96, 12, 14, 97, 98, 99, 100]
[[1, 38, 8, 9], [38, 2, 9, 39], [2, 8, 39, 40], [8, 9, 40, 41], [9, 39, 41, 2], [39, 40, 2, 42], [40, 41, 42, 13], [41, 2, 13, 1], [2, 42, 1, 43], [42, 13, 43, 23], [3, 44, 24, 45], [44, 11, 45, 46], [11, 24, 46, 47], [24, 45, 47, 1], [45, 46, 1, 14], [46, 47, 14, 25], [47, 1, 25, 48], [1, 14, 48, 10], [14, 25, 10, 26], [25, 48, 26, 2], [48, 10, 2, 27], [10, 26, 27, 12], [26, 2, 12, 11], [2,

In [22]:
for i in range(5):
  target = idx2word.get(targets[i])
  wrd = []
  for j in contexts[i]:
    wrd.append(idx2word.get(j))
  print(wrd,'->',target)

['the', 'speed', 'transmission', 'is'] -> of
['speed', 'of', 'is', 'an'] -> transmission
['of', 'transmission', 'an', 'important'] -> is
['transmission', 'is', 'important', 'point'] -> an
['is', 'an', 'point', 'of'] -> important


In [23]:
x = np.array(contexts)
y = np.array(targets)

In [24]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2)

In [46]:
model = Sequential([
    Embedding(input_dim=vocab_size,output_dim=emb_size,input_length=context_size),
    Lambda(lambda x:tf.reduce_mean(x,axis=1)),
    Dense(128,activation='relu'),
    Dense(128,activation='relu'),
    Dense(vocab_size,activation='softmax')
])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [47]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [48]:
H = model.fit(x_train,y_train,epochs=10,validation_data=(x_test,y_test))

Epoch 1/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 59ms/step - accuracy: 0.0026 - loss: 4.6342 - val_accuracy: 0.0000e+00 - val_loss: 4.6355
Epoch 2/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.1174 - loss: 4.6174 - val_accuracy: 0.0000e+00 - val_loss: 4.6356
Epoch 3/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.1135 - loss: 4.6030 - val_accuracy: 0.0000e+00 - val_loss: 4.6360
Epoch 4/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.0900 - loss: 4.5893 - val_accuracy: 0.0000e+00 - val_loss: 4.6367
Epoch 5/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.1000 - loss: 4.5709 - val_accuracy: 0.0000e+00 - val_loss: 4.6384
Epoch 6/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.1272 - loss: 4.5423 - val_accuracy: 0.0000e+00 - val_loss: 4.6401
Epoch 7/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.1368 - loss: 4.5074 - val_accuracy: 0.0000e+00 - val_loss: 4.6450
Epoch 8/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.1229 - loss: 4.4641 - val_accuracy

In [61]:
y_predicted = np.argmax(model.predict(np.array([[66, 22, 31, 68]])))
y_predicted

target = idx2word.get(y_predicted)
words = []
for i in [66, 22, 31, 68]:
  words.append(idx2word.get(i))
print(words, '->', target)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
['we', 'are', 'that', 'there'] -> the


In [55]:
y_test

array([ 1, 72, 14, 39, 10, 67, 68, 37, 81, 10, 18, 14, 21, 16, 59, 63, 33,
        9, 11, 20, 58, 28, 54, 28, 22, 26,  9, 17,  3, 88, 20, 34, 11])

In [56]:
x_test

array([[36, 21, 36,  2],
       [ 5,  6, 73, 74],
       [47,  1, 25, 48],
       [ 8,  9, 40, 41],
       [ 9, 90, 18, 13],
       [66, 22, 31, 68],
       [67, 31, 22, 69],
       [18, 13, 12, 37],
       [30, 80, 82, 10],
       [25, 48, 26,  2],
       [90, 10, 13, 37],
       [96, 12, 97, 98],
       [88, 89,  9, 90],
       [ 1, 15,  7,  4],
       [ 2, 58, 60, 61],
       [62,  8,  2,  1],
       [56,  8,  1, 57],
       [89, 21, 90, 10],
       [21,  9, 34, 35],
       [29, 19,  2, 58],
       [20,  2, 59, 60],
       [52, 20,  7,  3],
       [ 3, 32, 55, 17],
       [33, 65, 66, 22],
       [31, 68, 69, 70],
       [64,  1,  2, 27],
       [15, 16, 29, 20],
       [49, 50,  4,  5],
       [ 5, 12, 23, 22],
       [25, 87, 89, 21],
       [19, 52, 28,  7],
       [18, 11, 35,  2],
       [27, 12, 24, 15]])